In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
from dateutil.rrule import rrule, DAILY
from datetime import datetime
from dateutil import tz
from __future__ import division
import geoplotlib as glp
from geoplotlib.utils import BoundingBox, DataAccessObject

pd.set_option('display.max_columns', None)
%matplotlib inline  

# NYC Traffic and Weather - Explainer Notebook
*For DTU course [Social data analysis and visualization (02806)](http://www.kurser.dtu.dk/2013-2014/02806.aspx?menulanguage=en-GB)*

*Project Assignment B*

## Motivation
> What is your dataset?

Our dataset is [NYPD Motor Vehicle Collisions](https://data.cityofnewyork.us/Public-Safety/NYPD-Motor-Vehicle-Collisions/h9gi-nx95). It contains records for every reported incident in the NYC area. Records are available from July 2012 till today. Specifically, where the collision took place, the cause of the collision, injuries, fatalities and more.

Thhe other dataset used was an extraction of weather conditions for the NYC area. This was pulled from [Weather Underground](https://www.wunderground.com) using their csv service for hourly updates. For each hour, there are measurements of the temperature, visibility, windspeed as well as the overall weather conditions such as Rain, Snow, Clear etc.

> Why did you choose this/these particular dataset(s)?

Choosing the collisions dataset, was mainly out of interest in finding out where, how and why collisions happen.

The weather dataset was an afterthough and was mainly something we wanted to investigate after having looked collisions and the road conditions / traffic infrastructure.

> What was your goal for the end user's experience?

The main goal is to provide the user/reader with knowledge about how the weather affects the road and in that sense the collision rate. We would assume that more collisions happens in bad conditions, but can we quantify that? The user should also end up with an idea of where incidents happens the most, maybe to give an idea for the government of NYC for improvements. 

## Basic Stats. Let's understand the dataset better
>Write about your choices in data cleaning and preprocessing

Talking about the collisions dataset, the cleaning was mainly done for the columns that we knew we wanted to investigate or otherwise we though was important in the data exploration. Notably we wanted to make sure that we had location data for all the rows we investigated.

The weather information was not directly available to us and required a lot of HTTP requests to Weather Undergrounds servers. Below is how we did it:

In [ ]:
# Getting weather data from wunderground
start_date = date(2012, 7, 1)
end_date = date(2016, 2, 29)

# csv container for all daily weather infromation
frames = []

# url template for http requests. %s/%s/%s represent year/month/day
url_template = 'https://www.wunderground.com/history/airport/KJFK/%s/%s/%s/DailyHistory.html?req_city=New+York&req_state=NY&req_statename=New+York&reqdb.zip=10001&reqdb.magic=4&reqdb.wmo=99999&format=1.csv'
month = ""

# Query wunderground for daily weather information (returned as csv)
for dt in rrule(DAILY, dtstart=start_date, until=end_date):
    if (month != dt.strftime("%m")):
        month = dt.strftime("%m")
        print 'Downloading to memory: ' + dt.strftime("%Y-%m")
    # download and append csv file to csv container
    frames.append(pd.read_csv(url_template % (dt.strftime("%Y"),dt.strftime("%m"), dt.strftime("%d"))))

# Combine all csv's to one big and save it
print "Saving data to csv..."
data = pd.concat(frames)
data.to_csv('weather_data_nyc_kjfk.csv', sep=',')

We now have two datasets. Collisions and weather. However to avoid having to lookup in in a secondary dataset, that is the weather information, we merged the two datasets together. For the most part we had weather data for each hour for all the rows we wanted to investigate, with only a combined gap of a couple of days. In additions some hours had more than one row of weather information. We ignored these factors as we saw them insignificant to the overall result anyways.

In order to join the datasets they both had to have some columns in common. Which needed to be the date and time (hour). The weather dataset already had a datetime column in UTC. What we did was convert to NYC local time and add the columns Year, Month, Day, and Hour:

In [ ]:
# Read downloaded dataset
weather = pd.read_csv('datasets/weather_data_nyc_kjfk.csv')

# Convert UTC time to NYC actual
def UTCtoActual(utcDate):
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/New_York')
    
    utc = datetime.strptime(utcDate.DateUTC, '%Y-%m-%d %H:%M:%S')\
                  .replace(tzinfo=from_zone)\
                  .astimezone(to_zone)
    s = pd.Series([utc.year, utc.month, utc.day, utc.hour])
    s.columns = ['Year', 'Month', 'Day', 'Hour']
    return s

# Apply the above function to every row in the weather dataset and save the file.
weather[['Year', 'Month', 'Day', 'Hour']] = weather.apply(UTCtoActual, axis=1)
weather.to_csv('datasets/weather_data_nyc_kjfk_clean.csv')

With both datasets now having a 'common' ground for joining. This can now be done:

In [ ]:
# Read the datasets to be merged
incidents = pd.read_csv('datasets/NYPD_Motor_Vehicle_Collisions.csv')
weather = pd.read_csv('datasets/weather_data_nyc_kjfk_clean2.csv')

# Features from the dataset to merge in
features = ['Conditions', 'Precipitationmm', \
            'TemperatureC', 'VisibilityKm']

# Looks up weather data on the date and hour requested
def lookup_weather2(year, month, day, hour):
    w = weather[(weather.Year == year) & (weather.Month == month) & (weather.Day == day) & (weather.Hour == hour)]
    return w

# Looks up weather data, if nothing is found on the hour, it either looks an hour ahead or back.
# Otherwise returns empty.
def lookup_weather(date, time):
    month = int(date.split('/')[0])
    day = int(date.split('/')[1])
    year = int(date.split('/')[2])
    hour = int(time.split(':')[0])
    d = lookup_weather2(year, month, day, hour).head(1)
    if (d.empty):
        dt_back = datetime.datetime(year, month, day, hour) - datetime.timedelta(hours=1)
        dt_forward = datetime.datetime(year, month, day, hour) + datetime.timedelta(hours=1)
        
        d_back = lookup_weather2(dt_back.year, dt_back.month, dt_back.day, dt_back.hour)
        if (not d_back.empty): return d_back
        
        d_forward = lookup_weather2(dt_forward.year, dt_forward.month, dt_forward.day, dt_forward.hour)
        if (not d_forward.empty): return d_forward
    return d

# Merges the datasets
def merge_weather(incident):
    date = incident.DATE
    time = incident.TIME

    w = lookup_weather(date, time)

    try:
        # Default values
        con = "-"
        temp = "-"
        rainmm = "-"
        viskm = "-"

        # If weather data is different from null
        if (not pd.isnull(w['Conditions'].iloc[0])):
            con = w['Conditions'].iloc[0]
        if (not pd.isnull(w['TemperatureC'].iloc[0])):
            temp = w['TemperatureC'].iloc[0]
        if (not pd.isnull(w['Precipitationmm'].iloc[0])):
            rainmm = w['Precipitationmm'].iloc[0]
        if (not pd.isnull(w['VisibilityKm'].iloc[0])):
            viskm = w['VisibilityKm'].iloc[0]
            
        s = pd.Series([con, rainmm, temp, viskm])
        return s
    except:
        print date + "x" + time
        s = pd.Series([None,None,None,None])
        return s
    
print "Applying weather data to incidents..."
incidents[features] = incidents[incidents.DATE.str.split('/').str.get(2) != '2016'].apply(merge_weather, axis=1)
print "Saving weather in-riched incident data..."
incidents.to_csv('datasets/NYPD_Motor_Vehicle_Collisions_Weather_FINAL.csv', sep=',')

This was how we preprocessed and got all the available information that we require for making our data analysis.

>Write a short section that discusses the dataset stats (here you can recycle the work you did for Project Assignment A)


## Theory. Which theoretical tools did you use?

> Describe which machine learning tools you use and why the tools you've chosen are right for the problem you're solving.

Our motivation for investigating the collisions dataset was mainly to see if we would be able to find out where, how and why collisions happen in NYC. Could we predict where accidents are likely to happen? In class we have discussed two methods for classifiers that could be used to predict where the next crime was likely to happen. That is, KNN and Decision Trees.

...

In addition we used K-Means for clustering all incidents by location which data was used to power the decision tree. More on this later.

> Talk about your model selection. How did you split the data in to test/training. Did you use cross validation?

With around 650,000 rows to be split between test and training we were not really concered with too high variance. In the performance section we can talk about other splits and their results.

From the get go we know that finding the exact intersection where a collision is predictively going to take place is unfeasible to say the least. Doing some initial exploration using a trained decision tree targetting a longitude, latitude tuple we found an accuracy of 4-5%. 

If we cannot with great accuracy look at individual intersections we could look at zip codes. We could ask the question; in which postal area should we put an ambulance, given the weather and hour of day? Still the accuracy is  low. One way to improve the accuracy would be to add additional non-correlated features. Using K-Means to make location group-based clustering, that is longitude and latitude, how does that improve the model?

As mentioned we are using of using sklearns [DecisionTreeClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) we ended up using the [RandomForestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). We will be starting out with training a random forest of 50 trees. This should prevent any overfitting.

**In summary we are training a random forest of 50 trees, targetting/predicting/classifying ```ZIP CODE```, by using the features, ```KMEANS (location-based-clusters)```, ```HOUR```, ```Condition```.**

> Explain the model performance. How did you measure it? Are your results what you expected?

The model performance was measured lettings the decision tree classify the test data. Each row in of itself already has the correct ZIP CODE. By letting the tree predict the ZIP CODE and comparing with the actual value we can test if the prediction is correct. Doing that over the entire set of test data we can find a percentage accuracy of the model.

In [ ]:
# Test tree
def test_tree(clf, test_data, features):
    return clf.predict(test_data[features])

# Tests test_data on the tree classifier clf with features and target_label.
# encoded_map is a lookup table of the encoded values (numeric) to actual string values
def test_prediction(target_label, clf, test_data, features, encoded_map):
    corrects = 0
    predictions = test_tree(clf, test_data[features], features)
    for i in range(0, len(predictions)):
        if predictions[i] == test_data.iloc[i][target_label]:
            corrects += 1
    print "FOUND %d CORRECT PREDICTIONS" % corrects
    # Return model accuracy [%]
    return corrects / len(predictions)

# Test the decision tree
print "Prediction accuracy %f" % test_prediction(target_label_encoded, clf, test_data, features, target)